In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from copy import deepcopy
from collections import deque
from glob import glob
from io import StringIO, BytesIO
import json
from functools import partial


from scipy.spatial import cKDTree
import h5py
import k3d
import yaml
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pymesh
import torch
import trimesh.transformations as tt
import trimesh
import xml.etree.ElementTree as ET

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-q6l6z_5m because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [7]:
import os
os.environ['OMP_NUM_THREADS'] = '8'

In [65]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768,
                      cmap=k3d.colormaps.matplotlib_color_maps.coolwarm_r):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 1.0

            colors = k3d.helpers.map_colors(
                samples_distances, cmap, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            if samples_color == 'z':
                v = -np.array([0., 0., 1.])
                max_dist = np.max(np.dot(samples, v))
                min_dist = np.min(np.dot(samples, v))
                colors = k3d.helpers.map_colors(
                    np.dot(samples, v), k3d.colormaps.matplotlib_color_maps.viridis, [min_dist, max_dist]
                ).astype(np.uint32)
                k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
                
            else:
                k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
                
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [9]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [10]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.camera_pose_manager import POSE_MANAGER_BY_TYPE
from sharpf.data.imaging import IMAGING_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.utils.abc_utils.abc import feature_utils
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.plotting import display_depth_sharpness
from sharpf.utils.camera_utils.camera_pose import camera_to_display
from sharpf.utils.abc_utils.mesh.indexing import reindex_zerobased, compute_relative_indexes
import sharpf.data.data_smells as smells
import sharpf.utils.abc_utils.hdf5.io_struct as io

In [153]:
import numpy

# https://github.com/corochann/chainer-pointnet/blob/master/chainer_pointnet/utils/sampling.py

def l2_norm(x, y):
    """Calculate l2 norm (distance) of `x` and `y`.
    Args:
        x (numpy.ndarray or cupy): (batch_size, num_point, coord_dim)
        y (numpy.ndarray): (batch_size, num_point, coord_dim)
    Returns (numpy.ndarray): (batch_size, num_point,)
    """
    return ((x - y) ** 2).sum(axis=2)


def farthest_point_sampling(pts, k, initial_idx=None, metrics=l2_norm,
                            skip_initial=False, indices_dtype=numpy.int32,
                            distances_dtype=numpy.float32):
    """Batch operation of farthest point sampling
    Code referenced from below link by @Graipher
    https://codereview.stackexchange.com/questions/179561/farthest-point-algorithm-in-python
    Args:
        pts (numpy.ndarray or cupy.ndarray): 2-dim array (num_point, coord_dim)
            or 3-dim array (batch_size, num_point, coord_dim)
            When input is 2-dim array, it is treated as 3-dim array with
            `batch_size=1`.
        k (int): number of points to sample
        initial_idx (int): initial index to start farthest point sampling.
            `None` indicates to sample from random index,
            in this case the returned value is not deterministic.
        metrics (callable): metrics function, indicates how to calc distance.
        skip_initial (bool): If True, initial point is skipped to store as
            farthest point. It stabilizes the function output.
        xp (numpy or cupy):
        indices_dtype (): dtype of output `indices`
        distances_dtype (): dtype of output `distances`
    Returns (tuple): `indices` and `distances`.
        indices (numpy.ndarray or cupy.ndarray): 2-dim array (batch_size, k, )
            indices of sampled farthest points.
            `pts[indices[i, j]]` represents `i-th` batch element of `j-th`
            farthest point.
        distances (numpy.ndarray or cupy.ndarray): 3-dim array
            (batch_size, k, num_point)
    """
    if pts.ndim == 2:
        # insert batch_size axis
        pts = pts[None, ...]
    assert pts.ndim == 3
    xp = np
    batch_size, num_point, coord_dim = pts.shape
    indices = xp.zeros((batch_size, k, ), dtype=indices_dtype)

    # distances[bs, i, j] is distance between i-th farthest point `pts[bs, i]`
    # and j-th input point `pts[bs, j]`.
    distances = xp.zeros((batch_size, k, num_point), dtype=distances_dtype)
    if initial_idx is None:
        indices[:, 0] = xp.random.randint(len(pts))
    else:
        indices[:, 0] = initial_idx

    batch_indices = xp.arange(batch_size)
    farthest_point = pts[batch_indices, indices[:, 0]]
    # minimum distances to the sampled farthest point
    try:
        min_distances = metrics(farthest_point[:, None, :], pts)
    except Exception as e:
        import IPython; IPython.embed()

    if skip_initial:
        # Override 0-th `indices` by the farthest point of `initial_idx`
        indices[:, 0] = xp.argmax(min_distances, axis=1)
        farthest_point = pts[batch_indices, indices[:, 0]]
        min_distances = metrics(farthest_point[:, None, :], pts)

    distances[:, 0, :] = min_distances
    for i in tqdm(range(1, k)):
        indices[:, i] = xp.argmax(min_distances, axis=1)
        farthest_point = pts[batch_indices, indices[:, i]]
        dist = metrics(farthest_point[:, None, :], pts)
        distances[:, i, :] = dist
        min_distances = xp.minimum(min_distances, dist)
    return indices, distances



In [154]:
def patch_using_bfs(nn_indexes, centroid_idx):
    patch_points = {centroid_idx}
    stack = deque()
    stack.append(centroid_idx)
    
    while len(stack) > 0 and len(patch_points) < 4096:
        point_idx = stack.popleft()        
        for i in nn_indexes[point_idx]:
            if i not in patch_points and i not in stack:
                patch_points.add(point_idx)
                stack.append(i)
#         print(len(stack), len(patch_points))

    return np.array(list(patch_points))


def patches_from_point_cloud(points, n_patches):
    # spread patch centroids across the point clouds
    inds, ds = farthest_point_sampling(points, k=n_patches)
    
    # get kNN graph from point cloud
    tree = cKDTree(points, leafsize=1024)
    nn_distances, nn_indexes = tree.query(points, k=5, n_jobs=10)
    
    # startting from each seed centroid, traverse kNN graph and yield patches (slow)
    for centroid_idx in tqdm(inds[0]):
        yield patch_using_bfs(nn_indexes, centroid_idx)

In [222]:
def load_scans(base_dir):
    matlab_project_filename = glob(os.path.join(base_dir, '*.mlp'))[0]
    root = ET.parse(matlab_project_filename).getroot()

    points_by_scan = []
    transform_by_scan_4x4 = []

    item_id = None
    for type_tag in root.findall('MeshGroup/MLMesh'):
        filename = type_tag.get('filename')
        if filename.endswith('.obj'):
            item_id = filename

        elif filename.endswith('.ply'):
            try:
                transform = np.loadtxt(
                    StringIO(type_tag.find('MLMatrix44').text))
                points = trimesh.load(os.path.join(base_dir, filename)).vertices
            except ValueError as e:
                continue
            
            transform_by_scan_4x4.append(transform)
            points_by_scan.append(points)
            
    return points_by_scan, transform_by_scan_4x4, item_id

In [156]:
UnlabeledPointCloudIO = io.HDF5IO({
        'points': io.Float64('points'),
        'indexes_in_whole': io.Int32('indexes_in_whole'),
        'distances': io.Float64('distances'),
        'item_id': io.AsciiString('item_id'),
    },
    len_label='has_sharp',
    compression='lzf')


def save_point_crops(patches, filename):
    # turn a list of dicts into a dict of torch tensors:
    # default_collate([{'a': 'str1', 'x': np.random.normal()}, {'a': 'str2', 'x': np.random.normal()}])
    # Out[26]: {'a': ['str1', 'str2'], 'x': tensor([0.4252, 0.1414], dtype=torch.float64)}
    collate_fn = partial(io.collate_mapping_with_io, io=UnlabeledPointCloudIO)
    patches = collate_fn(patches)

    with h5py.File(filename, 'w') as f:
        for key in ['points', 'indexes_in_whole', 'distances']:
            UnlabeledPointCloudIO.write(f, key, patches[key].numpy())
        UnlabeledPointCloudIO.write(f, 'item_id', patches['item_id'])

In [163]:
def process_scans(input_dir, output_filename):
    points_by_scan, transform_by_scan_4x4, item_id = load_scans(input_dir)
    
    all_points = np.concatenate([
        tt.transform_points(points, transform)
        for points, transform in zip(points_by_scan, transform_by_scan_4x4)])
    n_patches = int(len(all_points) * 10 / 4096)
    print('n_patches = ', n_patches)
    
    point_patches = []
    for patch_point_indexes in patches_from_point_cloud(all_points, n_patches):
        if len(patch_point_indexes) == 4096:
            point_patches.append({
                'points': all_points[patch_point_indexes],
                'distances': np.ones(4096),
                'indexes_in_whole': patch_point_indexes,
                'item_id': item_id
            })
            
    save_point_crops(point_patches, output_filename)
    return all_points, point_patches

In [212]:
s = !find /data/abc/sharp_features_whole_models/SL -maxdepth 1 -mindepth 1 -type d 

In [213]:
todo = [os.path.basename(d) for d in s
       if not os.path.exists(
           '/data/abc/sharp_features_whole_models/SL/{}/{}.hdf5'.format(os.path.basename(d), os.path.basename(d)))]

In [223]:
todo = [
#     'smallest_345',
 'bat_320',
 'smallest_40',
 'small_40',
 'cross_40',
 'Detail_45',
 'car_320',
 'cube_small_40',
 'hook_40cm']

In [224]:
for name in todo:
    print(name)
    all_points, point_patches = process_scans(
        '/data/abc/sharp_features_whole_models/SL/{}/'.format(name),
        '/data/abc/sharp_features_whole_models/SL/{}/{}.hdf5'.format(name, name)
    )

bat_320


100%|██████████| 92/92 [00:00<00:00, 548.64it/s]

n_patches =  93



100%|██████████| 93/93 [00:09<00:00, 10.04it/s]


smallest_40


...

100%|██████████| 1084/1084 [02:19<00:00,  7.75it/s]


cross_40
n_patches =  7325


100%|██████████| 7325/7325 [17:20<00:00,  7.04it/s]


Detail_45
n_patches =  28178


MemoryError: Unable to allocate 1.18 TiB for an array with shape (1, 28178, 11541725) and data type float32

In [184]:
# points_by_scan, transform_by_scan_4x4, item_id = load_scans('/data/abc/sharp_features_whole_models/SL/Castle_45/')

# all_points = np.concatenate([
#     tt.transform_points(points, transform)
#     for points, transform in zip(points_by_scan, transform_by_scan_4x4)])

# display_sharpness(samples=all_points, samples_psize=0.1, samples_color='z')

In [197]:
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes 

ground_truth = Hdf5File('/data/abc/sharp_features_whole_models/SL/Cross_345/Cross_345.hdf5',
                   io=UnlabeledPointCloudIO,
                   preload=PreloadTypes.LAZY,
                   labels='*')


n_points = np.concatenate([patch['indexes_in_whole'] for patch in ground_truth]).max() + 1
whole_model_points_gt = np.zeros((n_points, 3))
whole_model_distances_gt = np.ones(n_points) * np.inf

for patch in tqdm(ground_truth):
    distances = patch['distances']
    indexes = patch['indexes_in_whole']
    whole_model_points_gt[indexes] = patch['points'].reshape((-1, 3))

    assign_mask = whole_model_distances_gt[indexes] > distances
    whole_model_distances_gt[indexes[assign_mask]] = np.minimum(distances[assign_mask], 1.0)


    
display_sharpness(samples=whole_model_points_gt, samples_psize=0.1, samples_color='z')

File /data/abc/sharp_features_whole_models/SL/Cross_345/Cross_345.hdf5 is not compatible with Hdf5File I/O interface <class 'sharpf.utils.abc_utils.hdf5.io_struct.HDF5IO'>
File /data/abc/sharp_features_whole_models/SL/Cross_345/Cross_345.hdf5 is not compatible with Hdf5File I/O interface <class 'sharpf.utils.abc_utils.hdf5.io_struct.HDF5IO'>
77it [00:00, 6802.77it/s]


Output()

In [2]:
import trimesh

In [14]:
scan = trimesh.load('/data/abc/sharp_features_whole_models/scan_res_0000/AlexeyDetail_Big 1.ply')

In [55]:
scan.vertices

TrackedArray([[  -1.06496   , -143.62800598,   93.68890381],
              [  -5.25204992, -137.04899597,   93.61959839],
              [ -30.73900032, -142.18400574,   79.54850006],
              ...,
              [ -31.39830017, -138.03300476,   78.66459656],
              [   5.01087999, -151.92399597,   91.376297  ],
              [ -29.00819969, -160.59500122,   81.30799866]])

In [66]:
display_sharpness(
    samples=scan.vertices,
    samples_color='z',
    samples_psize=0.05,
    sharp_vert=[[0.,0.,0.0]],
    sharpvert_psize=10.0
)

Output()

In [27]:
M = np.loadtxt('/data/abc/sharp_features_whole_models/scan_res_0000/vertex_matrix.txt')

In [39]:
M.tolist()

[[0.978958, -3.72529e-09, 0.20406, 0.0],
 [-0.0573489, 0.959696, 0.275126, 0.0],
 [-0.195835, -0.28104, 0.939503, 0.0],
 [0.0, 0.0, 0.0, 1.0]]

In [30]:
with open('/data/abc/sharp_features_whole_models/scan_res_0000/Raw/impara01.txt') as f:
    impara01 = f.read().strip().split('\n')


In [32]:
angles = np.array([0.22859, 0.188937, -0.000396054])

In [41]:
X0 = np.array([-233.672, -335.338, 1121.02])

In [37]:
def rotation_matrix(angles):
    angle_x, angle_y, angle_z = angles
    e_x, e_y, e_z = np.eye(3)

    rotation_x = tt.rotation_matrix(angle_x, e_x)
    rotation_y = tt.rotation_matrix(angle_y, e_y)
    rotation_z = tt.rotation_matrix(angle_z, e_z)

    rotation = np.dot(
        np.dot(rotation_x, rotation_y),
        rotation_z)

    return rotation

In [46]:
T = np.dot(
    rotation_matrix(angles),
    tt.translation_matrix(X0))

In [56]:
display_sharpness(
    samples=tt.transform_points(scan.vertices, T),
    samples_color='z',
    samples_psize=0.05,
    sharp_vert=[X0],
    sharpvert_psize=10.0)

Output()